In [36]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time, random

import json
import re
import os

job_links = []

# Header giả lập trình duyệt thật
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    # 👇 Tắt yêu cầu nén gzip/br
    "Accept-Encoding": "identity",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
}

def crawl_page(page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    try:
        # Delay ngẫu nhiên tránh bị chặn
        time.sleep(random.uniform(2, 4))

        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = response.apparent_encoding  # tự đoán encoding

        # Lưu HTML của trang ra file để kiểm tra
        #with open(f"page_{page}.html", "wb") as f:
            #f.write(response.content)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            all_job = soup.find_all("div", class_="job-item-2")
            links = []
            for job in all_job:
                a_tag = job.find("a", href=True)
                if a_tag:
                    links.append(a_tag["href"])
            # print(f"Trang {page}: {len(links)} links")
            return links
        else:
            print(f"Trang {page} lỗi HTTP {response.status_code}")
    except Exception as e:
        print(f"Lỗi trang {page}: {e}")
    return []


# === Chạy đa luồng nhẹ (đừng quá cao nếu không bị chặn) ===
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(crawl_page, range(2, 22))  # thử trước 20 trang
    for r in results:
        job_links.extend(r)

print("\nTổng số link thu được:", len(job_links))


Tổng số link thu được: 1000


In [37]:
job_links[0]

'https://www.topcv.vn/viec-lam/executive-it-support/1926095.html?ta_source=ITJobs_LinkDetail'

In [ ]:
def link_to_json(url,json_file_name):
    
    #Lấy soup
    response = requests.get(url, headers= headers, timeout = 15)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        # print(soup)

    #Lấy job_description_content
    job_description_content = soup.find("div", class_=lambda c: c and "job-description" in c)
    # print(job_description_content)
    job_text = job_description_content.get_text(separator="\n",strip=True)
    # print(job_text)

    # text = extract_job_info(soup)
    output_lines = []
    print("=== Tên công ty ===")
    company_name_label = soup.find("div", class_ = "company-name-label")
    if ( company_name_label ):
        a_tag = company_name_label.find("a")
        if a_tag:
            company_name = a_tag.get_text(strip=True)
            output_lines.append("=== Tên công ty ===")
            output_lines.append(company_name)

    job_detail_info = soup.find_all("div", class_="job-detail__info--section")
    for section in job_detail_info:
        title_tag = section.find("div", class_="job-detail__info--section-content-title")
        value_tag = section.find("div", class_="job-detail__info--section-content-value")

        if value_tag:
            a_tag = value_tag.find("a")
            if a_tag:
                value_text = a_tag.get_text(strip=True)
            else:
                value_text = value_tag.get_text(strip=True)
        else:
            value_text = ""

        title_text = title_tag.get_text(strip=True) if title_tag else "Không có tiêu đề"

        output_lines.append(f"=== {title_text} ===")
        output_lines.append(value_text)

    items = job_description_content.find_all("div", class_ = "job-description__item")
    for item in items:
        title = item.find("h3").get_text(strip=True)
        content = item.find("div", class_="job-description__item--content").get_text(separator="\n", strip=True)
        output_lines.append(f"=== {title} ===")
        output_lines.append(content)
    
    text = "\n".join(output_lines)
    # text_to_json(text, TÊN_FILE_Ở_ĐÂY)
    # Oke, tới đây đã lấy được text rồi nè
    # print(text)
    pattern = r"===\s*(.*?)\s*===\s*([\s\S]*?)(?===|$)"
    matches = re.findall(pattern, text)

    job_info = {}
    for title, content in matches:
        clean_title = title.strip()
        clean_content = content.strip()
        job_info[clean_title] = clean_content

    #Đọc file hiện có
    filename = json_file_name
    if (os.path.exists(filename)):
        with open(filename, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []
    else:
        data = []

    data.append(job_info)

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"Đã thêm một bản ghi vào {filename}")
    return job_info

In [39]:
url = job_links[0]
print(url)

https://www.topcv.vn/viec-lam/executive-it-support/1926095.html?ta_source=ITJobs_LinkDetail


In [35]:
link_to_json(url,"jobs.json")

=== Tên công ty ===
Đã thêm một bản ghi vào jobs.json


{'Tên công ty': 'Expolanka Freight (Vietnam) Ltd',
 'Mức lương': 'Thoả thuận',
 'Địa điểm': 'Hồ Chí Minh',
 'Kinh nghiệm': '1 năm',
 'Mô tả công việc': 'Key Responsibilities:\n· Install, configure and maintain overall policies\n· Manage daily operations of IT infrastructure including servers and network.\n· Check performance/operation status of servers, network devices, security system.\n· Requests create / remove Account (email & ERP system) for new staff/ leavers.\n· Manage software license and update.\n· Provide daily support to end users on a variety of issues (internet, email, applications, printing....)\n· Identify, research and resolve IT technical problems.\n· Monitor and troubleshoot hardware and software problems.\n· Set-up, configure laptop/PCs for new staff (installation of relevant software, printers....)\n· Support to technically manage office equipment (printers, projectors, copy machines, telephone operator...).\n· Assist on data extraction on ERP system, creating & sch

In [40]:
for i in range(1, 1001):
    # Tính số thứ tự file (chia 100, làm tròn lên)
    number_of_file = (i - 1) // 100 + 1

    # Tạo tên file tương ứng
    file_name = f"job_data_{number_of_file}.json"

    # Gọi hàm lưu dữ liệu
    link_to_json(job_links[i], file_name)

    print(f"Đã lưu job {i} vào file {file_name}")


=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 1 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 2 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 3 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 4 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 5 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 6 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 7 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 8 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 9 vào file job_data_1.json
=== Tên công ty ===
Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 10 vào file job_data_1.json
=== Tên công ty ===

UnboundLocalError: cannot access local variable 'soup' where it is not associated with a value